In [ ]:
import os
# ####################################### SET PARAMETERS ####################################
EXPERIMENTS_ROOT = "../data/experiments/"

EXPERIMENT_1_DIR = "shila"
EXPERIMENT_1_NAME = "Shila"
EXPERIMENT_1_AREA_KEY = "Area um^2" #"DAPI: DAPI area um^2"
EXPERIMENT_1_ROOT = os.path.join(EXPERIMENTS_ROOT, EXPERIMENT_1_DIR)
EXPERIMENT_1_ANIMALS_ROOT = os.path.join(EXPERIMENT_1_ROOT, "QuPath_output")
EXPERIMENT_1_DIRS = ["16C", "17C", "19C", "5S", "8S", "10S", "13S", "14S", "1R", "2R", "3R", "4R", "11R"]

EXPERIMENT_2_DIR = "soumnya/experiment"
EXPERIMENT_2_NAME = "Soumnya"
EXPERIMENT_2_AREA_KEY = "Area um^2"
EXPERIMENT_2_ROOT = os.path.join(EXPERIMENTS_ROOT, EXPERIMENT_2_DIR)
EXPERIMENT_2_ANIMALS_ROOT = os.path.join(EXPERIMENT_2_ROOT, "QuPath_output")
EXPERIMENT_2_DIRS = ["41CM", "42CF", "43CM", "44CF", "49CM", "50CF", "51CM", "52CF", "58CF", "60CF", "74CF", "76CF", "81CM", "83CM", "89CM", "91CM", "45SM", "46SF", "47SM", "48SF", "53SM", "54SF", "55SM", "56SF", "62SF", "64SF", "78SF", "80SF", "85SM", "87SM", "93SM", "95SM"]
# empty results:
# - Control_51C

TRACER_KEY = "Num AF647"
MARKER = "CFos"

In [ ]:
import sys

PROJECT_PATH = os.path.dirname(os.path.abspath(os.getcwd()))
sys.path.append(PROJECT_PATH)
import BraiAn

In [ ]:
# from https://help.brain-map.org/display/api/Downloading+an+Ontology%27s+Structure+Graph
# StructureGraph id=1
path_to_allen_json = os.path.join(PROJECT_PATH, "data", "AllenMouseBrainOntology.json")
BraiAn.cache(path_to_allen_json, "http://api.brain-map.org/api/v2/structure_graph_download/1.json")
branches_to_exclude = ["retina","VS","grv","fiber tracts"]
brain_onthology = BraiAn.AllenBrainHierarchy(path_to_allen_json, branches_to_exclude)

#parent_region = brain_onthology.parent_region
#direct_subregions = brain_onthology.direct_subregions
#full_name = brain_onthology.full_name
#regions = list(full_name.keys())

In [ ]:
slice_root = f"{EXPERIMENT_1_ANIMALS_ROOT}/{EXPERIMENT_1_DIRS[2]}"
image = "2022_04_22__12_22__2525.czi - Scene #01"
slice = BraiAn.BrainSlice(brain_onthology,
                    f"{slice_root}/results/{image}_regions.txt",
                    f"{slice_root}/regions_to_exclude/{image}_regions_to_exclude.txt",
                    EXPERIMENT_1_DIRS[2], image, EXPERIMENT_1_AREA_KEY, TRACER_KEY, MARKER)
slice.data

In [ ]:
experiment_1_slices = [BraiAn.SlicedBrain(animal_dir,
                                        os.path.join(EXPERIMENT_1_ANIMALS_ROOT, animal_dir),
                                        brain_onthology,
                                        EXPERIMENT_1_AREA_KEY,
                                        TRACER_KEY,
                                        MARKER,
                                        area_units="µm2",
                                        exclude_parent_regions=False)
                    for animal_dir in EXPERIMENT_1_DIRS]
print(f"NOTE: Imported all brain slices from {str(len(experiment_1_slices))} animals of {EXPERIMENT_1_NAME} experiment.")

experiment_2_slices = [BraiAn.SlicedBrain(animal_dir,
                                        os.path.join(EXPERIMENT_2_ANIMALS_ROOT, animal_dir),
                                        brain_onthology,
                                        EXPERIMENT_2_AREA_KEY,
                                        TRACER_KEY,
                                        MARKER,
                                        area_units="µm2",
                                        exclude_parent_regions=False)
                    for animal_dir in EXPERIMENT_2_DIRS]
print(f"NOTE: Imported all brain slices  {str(len(experiment_2_slices))} animals of {EXPERIMENT_2_NAME} experiment.")

In [ ]:
experiment_1_sum_brains = [BraiAn.AnimalBrain.from_slices(sliced_brain) for sliced_brain in experiment_1_slices if len(sliced_brain.slices) > 0]
experiment_2_sum_brains = [BraiAn.AnimalBrain.from_slices(sliced_brain) for sliced_brain in experiment_2_slices if len(sliced_brain.slices) > 0]

In [ ]:
for brain in experiment_1_sum_brains + experiment_2_sum_brains:
    if "Left: root" not in brain.data.index:
        print(f"{brain.name} has no 'Left: root' area")
    if "Right: root" not in brain.data.index:
        print(f"{brain.name} has no 'Right: root' area")

In [ ]:
experiment_1 = BraiAn.AnimalGroup(EXPERIMENT_1_NAME, experiment_1_sum_brains, brain_onthology)
experiment_2 = BraiAn.AnimalGroup(EXPERIMENT_2_NAME, experiment_2_sum_brains, brain_onthology)

In [ ]:
normalization = "Density"
threshold = 700 # Only plot bars with value larger than threshold
experiments = [experiment_1, experiment_2]
regions_to_plot = ["root"] #BraiAn.regions_to_plot(groups=groups, normalization=normalization, threshold=threshold)
fig = BraiAn.plot_groups(normalization, brain_onthology, experiment_1, experiment_2,
                            selected_regions=regions_to_plot, use_acronyms=False,
                            width=900, barheight=100)
fig.show()

file_title = f"barplot_{experiment_1.markers[0]}_{normalization}_{len(experiments)}groups.png"